<a href="https://colab.research.google.com/github/sudowoodo200/DeepSpeedRewardMod/blob/master/Conformal_Pred_vSL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Mostly loading of packages and huggingface login

In this notebook we aim to implement the techniques neccessary to detect a distribution shift on a Learning from human preference setting. We start loading the packages.

In [1]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [2]:
!nvidia-smi

Sun May 28 05:03:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install wandb
!pip install tqdm
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install torch
!pip install statsmodels
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=733accd940b335a108e773b9963d2dc3fd0ac296ea53e8bf085279028d3663a8
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/

In [5]:
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
import numpy as np
from datasets import load_metric
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from statsmodels.stats.multitest import multipletests
#from statsmodels.stats.weightstats import ttest_ind
from scipy.stats import binomtest, ttest_ind, ttest_1samp

In [6]:
# Log in to your Hugging Face account 
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

# Data loading

We ingest data from ```openai/tldr``` and first load it into a standardized format, then massage it to be compatible with the tokenizer. The model expects data in the following schema (length controlled and tokenized, but unpadded; padding is handled by the dataloader):

```
train:
  "x1": list[tokens] = title || post || summary_1,
  "x2": list[tokens] = title || post || summary_2,
  "labels" : 0 or 1
```
where x1 and x2 are bound by distil-bert's 512 token limit and tokenized appropriately.

## Basic Loading

Output: ```X_train, X_cal, X_shift``` are dataframes in the specified format

```
X_Train:
  "subreddit": str
  "title: str
  "post: str
  "summary_0": str
  "summary_1": str
  "label": 0 or 1
```

In [7]:
tldr = load_dataset("openai/summarize_from_feedback", "comparisons").shuffle()   # https://huggingface.co/datasets/openai/summarize_from_feedback

tldr_train = pd.DataFrame(tldr['train'])
tldr_train = tldr_train.replace(to_replace='None', value=np.nan).dropna()
tldr_train = tldr_train[:1000]
tldr_cal = pd.DataFrame(tldr['validation'])
tldr_cal = tldr_cal.replace(to_replace='None', value=np.nan).dropna()
tldr_cal = tldr_cal[:500]

slf5k = load_dataset("JeremyAlain/SLF5K") # the test dataset can be taken from https://huggingface.co/datasets/JeremyAlain/SLF5K
slf5k_test = pd.DataFrame(slf5k['validation'])

Extracting data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset summarize_from_feedback downloaded and prepared to /root/.cache/huggingface/datasets/openai___summarize_from_feedback/comparisons/0.0.0/483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Generating development split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset slf5_k downloaded and prepared to /root/.cache/huggingface/datasets/JeremyAlain___slf5_k/SLF5K/1.0.0/6b37f332eea04ffa072f2c66e87393132bd68a310796894ee18fb105544d3294. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
texts = []
for d in tldr_train['summaries']:
    texts.append([d[0]['text'], d[1]['text']])

summaries_df = pd.DataFrame(texts, columns=['summary_0', 'summary_1'])


X_train = pd.DataFrame({
    'subreddit': pd.DataFrame(list(tldr_train['info'].values))['subreddit'],
    'title': pd.DataFrame(list(tldr_train['info'].values))['title'],
    'post': pd.DataFrame(list(tldr_train['info'].values))['post'],
    'summary_0': summaries_df['summary_0'],
    'summary_1': summaries_df['summary_1'],
    'label': tldr_train['choice']
})

X_train.head()

,subreddit,title,post,summary_0,summary_1,label
0,relationships,I[23M] asked out a girl [20F] that I really li...,So there's this girl in my class that I met an...,"Asked a girl out, she said she just got out o...","I asked out a girl, she said that she just go...",1
1,relationships,Me [24 M] with my brother [21 F] He is in the ...,Hello people\n\nThis is really hard for me to ...,Brother is a sparkplug of a guy and wants to ...,My brother wants to serve in the Middle East ...,1
2,relationships,How do I get over heart break? Male (18),I'm sorry if this isn't the right subreddit fo...,Girlfriend cheated on me. Felt like a fool an...,"GF cheated on me, lied to me, and now I don't...",1
3,relationship_advice,[26f] On-going interaction with ex [26/m] has ...,I dated this guy for a few months after having...,Hooking up with ex every few months despite n...,long distance relationship has left me confus...,0
4,personalfinance,Paying student loans before graduating,I started grad school in January while working...,Should I pay off loans right away or wait unt...,"I have $10k in loans, should I start paying o...",0


In [9]:
texts = []
for d in tldr_cal['summaries']:
    texts.append([d[0]['text'], d[1]['text']])

summaries_df = pd.DataFrame(texts, columns=['summary_0', 'summary_1'])

X_cal = pd.DataFrame({
    'subreddit': pd.DataFrame(list(tldr_cal['info'].values))['subreddit'],
    'title': pd.DataFrame(list(tldr_cal['info'].values))['title'],
    'post': pd.DataFrame(list(tldr_cal['info'].values))['post'],
    'summary_0': summaries_df['summary_0'],
    'summary_1': summaries_df['summary_0'],
    'label': tldr_cal['choice']
})

X_cal.head()

,subreddit,title,post,summary_0,summary_1,label
0,self,18 no direction in life after dropping out of ...,"I'm 18, finished high school with great grades...","Dropped out of law school, passionate about e...","Dropped out of law school, passionate about e...",0
1,relationships,"Me [22M] with my ex as of 12 hours ago, [22F] ...",I met this girl just over a year ago at a univ...,"Long distance dating for 14 months, we both a...","Long distance dating for 14 months, we both a...",1
2,relationship_advice,(21f)Accidentally spilled out a drunken 'I thi...,My boyfriend and I just started dating a few d...,blurt out the L word drunk to brand new boyfr...,blurt out the L word drunk to brand new boyfr...,0
3,relationships,"Me [20 M] with my ___ [18F] duration, short-de...",last summer i fell in love for the first time ...,falling in love with GF but she doesnt want a...,falling in love with GF but she doesnt want a...,1
4,relationships,"Me[19 M] with my[23 F] friend who has a LDR, w...",I first met this girl through an mmo mobile ga...,My attractive friend wants to keep hanging ou...,My attractive friend wants to keep hanging ou...,1


Here we define the test/shift dataset. To make better simulate our target situation, we select just a small subset of size n.

In [10]:
X_shift = pd.DataFrame({
    'subreddit': slf5k_test['subreddit'],
    'title': slf5k_test['title'],
    'post': slf5k_test['post'],
    'summary_0': slf5k_test['generated_summary_for_comparison_A'],
    'summary_1': slf5k_test['generated_summary_for_comparison_B'],
    'label': ("Summary B" == slf5k_test['comparison_preference']).astype(int)
})

X_shift.head()

,subreddit,title,post,summary_0,summary_1,label
0,relationships,Me [23/F] with my ex-boyfriend[22/M] have been...,I've known my now ex boyfriend for over 10 yea...,My ex and I have been broken up for a few week...,Ex-boyfriend and I have been dating for a year...,1
1,relationships,Girlfriend [20F] wasting time and money in col...,GF: 20F\nMe: 22M\nLength of relationship: 8 mo...,"GF is 20F, studying premed and failing grades;...","Girlfriend 20F is studying premed, but is not ...",0
2,books,I just wrote a 200 page science fiction/fantas...,"I just sent the completed, unedited novel to t...","I wrote a novel, paid to register it with the ...",I'm a new author that has completed a 200 page...,0
3,legaladvice,Landlord sent us a cease and desist letter. Wh...,"Dear Reddit,\n\nUsing a throwaway account. Her...",Landlord sent us a cease and desist letter to ...,Landlord is angry we're moving out early and i...,1
4,relationships,My college SO and I [20M & 20F] are getting mo...,"My girlfriend and I are distant, but *we live ...","I'm feeling really distant from my SO, and it'...","GF is distant, I am hurt and angered. Please h...",0


## Model and tokenization

The utilities function is discretized into $k$ levels, bounded within the $[-1,1]$ range. The levels are then predicted using a BERT model. Here's the pseudocode for each data row

```
u_map = [-1,1; k steps]
for i = 0, 1:
  logit_i = BERT(text || x_i)    # in R^k
  rho_i = softmax(logit_i)       # in R^k
  utility_i = dot_product(rho_i, u_map)  # in R
select = sigmoid(utility_1 - utility_0)
loss = cross_entropy(select, choice_label)
```


Note that the model learns a weak ordering of the labels via the scalar utilities mapping and the preference selections. 

This requires both a symmetric data loader and a custom trainer, which will be jerry rigged below

In [11]:
# Set DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

### Reforming data for DataLoader

In [55]:
from typing import Dict, List, Any, Tuple
from pandas.core.series import Series
import numpy as np

X_cal = X_cal.replace(to_replace='None', value=np.nan).dropna()
X_train = X_train.replace(to_replace='None', value=np.nan).dropna()
X_shift = X_shift.replace(to_replace='None', value=np.nan).dropna()

def data_preprocessor(df: pd.DataFrame, 
                      shared_data_cols: List[str] = ["title", "post"],
                      labeled_data_cols: List[str] = ["summary_"],
                      labels: List[str] = ["label"],
                      max_length: int = 512,
                      tokenizer: Any = tokenizer) -> pd.DataFrame:

  def encode_and_concatenate(row: Series) -> List[int]:
    out = []
    for x in row:
      out += tokenizer.encode(x)
    return out

  def valid_and_check(x: Any) -> bool:
    if x is np.nan:
      return False
    else:
      return len(x) < max_length


  ## filter out rows that exceed the BERT context length limit.
  ## do not truncate to avoid biasing
  shared_prefix = df[shared_data_cols].apply(encode_and_concatenate, axis=1)
  out = pd.DataFrame()
  for k in [0,1]:
    labeled_cols = [x+str(k) for x in labeled_data_cols]
    out[k] = shared_prefix + df[labeled_cols].apply(encode_and_concatenate, axis=1)
    checks = out[k].apply(valid_and_check)
    shared_prefix = shared_prefix[checks]
    df = df.loc[checks]
    out = out.loc[checks]

  out.columns = ["input_ids_a", "input_ids_b"]
  out["attention_mask_a"] = out["input_ids_a"].apply(lambda x: [1]*len(x))
  out["attention_mask_b"] = out["input_ids_b"].apply(lambda x: [1]*len(x))
  out["labels"] = df[labels]

  return out

X_t = data_preprocessor(X_train)
X_c = data_preprocessor(X_cal)
X_s = data_preprocessor(X_shift)

print(X_t.shape)
print(X_c.shape)
print(X_s.shape)


(929, 5)
(451, 5)
(474, 5)


In [56]:
X_train_dataset = Dataset.from_pandas(X_t)
X_cal_dataset = Dataset.from_pandas(X_c)
X_shift_dataset = Dataset.from_pandas(X_s)

# Reward Model training

In [50]:
# Define DistilBERT as our base model:
k = 9
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=k) #, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [51]:
## Modifying DataCollatorWithPadding
## https://github.com/huggingface/transformers/blob/d95a32cc60e5d92b4bf08cd805c6b0db7b4100cc/src/transformers/data/data_collator.py#L212:L260

from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy
from typing import Union, Optional, List, Dict, Any
from dataclasses import dataclass

@dataclass
class SymmetricDataCollatorWithPadding:

  tokenizer: PreTrainedTokenizerBase # Tokenizer type
  padding: Union[bool, str, PaddingStrategy] = True
  max_length: Optional[int] = None
  pad_to_multiple_of: Optional[int] = None
  return_tensors: str = "pt"

  def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:

        feature_a = []
        feature_b = []
        for feature in features:
          feature_a.append({
              "labels": feature["labels"],
              "input_ids": feature["input_ids_a"],
              "attention_mask": feature["attention_mask_a"]
          })
          feature_b.append({
              "labels": feature["labels"],
              "input_ids": feature["input_ids_b"],
              "attention_mask": feature["attention_mask_b"]
          })

        batch_a = self.tokenizer.pad(
            feature_a,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )

        batch_b = self.tokenizer.pad(
            feature_b,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )

        batch = {
            "labels" : batch_a["labels"],
            "input_ids_a" : batch_a["input_ids"],
            "input_ids_b" : batch_b["input_ids"],
            "attention_mask_a" : batch_a["input_ids"],
            "attention_mask_b" : batch_b["input_ids"]
        }

        return batch

data_collator = SymmetricDataCollatorWithPadding(tokenizer=tokenizer)

In [53]:
import torch.nn.functional as F

# alternate trainer object to get reward model from object ranking
# https://huggingface.co/docs/transformers/main_classes/trainer
# https://github.com/lvwerra/trl/blob/3cfe194e34ee5fb4e2bcf0935e9a6dcd1eebea8e/trl/trainer/utils.py#L50
# https://github.com/lvwerra/trl/blob/3cfe194e34ee5fb4e2bcf0935e9a6dcd1eebea8e/trl/trainer/reward_trainer.py#L35

DELIMITER = [102, 101] # '[SEP][CLS]'
ENDER = [102, 102]

class CustomTrainer(Trainer):

  def __init__(self, *args, **kwargs):

    super().__init__(*args, **kwargs)
    self.k = k      # k = 9 by default. see cell above
    self.u_map = torch.linspace(-1.0, 1.0, self.k, device=model.device).unsqueeze(0)

  def compute_util(self, model, y, mask):

    rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
    return torch.sum( rho * self.u_map , dim = 1)


  def compute_loss(self, model, inputs, return_outputs=False):

    labels = inputs.get("labels")
    input_a = inputs.get("input_ids_a")
    input_b = inputs.get("input_ids_b")
    mask_a = inputs.get("attention_mask_a")
    mask_b = inputs.get("attention_mask_b")

    r1, r2 = [self.compute_util(model, y, mask) for (y, mask) in [(input_a, mask_a), (input_b, mask_b)]]
    p = F.sigmoid(r1 - r2)
    loss = F.binary_cross_entropy(p, labels.float())
    
    return loss


# Define accuracy and f1 as the metrics:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}


In [57]:
# Define a new Trainer with all the objects we constructed so far
repo_name = "summaries-comparisons-distilbert-TLDR"

import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=False,
    remove_unused_columns=False
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=X_train_dataset,
    eval_dataset=X_cal_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train and push to hub
torch.cuda.empty_cache()
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
Could not estimate the num

Step,Training Loss
500,0.672200


<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rho = F.softmax(model(input_ids = y, attention_mask = mask).get("logits"))
<ipython-input-53-3c62a041eeab>:21: UserWarning: Implicit dimens

TrainOutput(global_step=590, training_loss=0.6522459773694055, metrics={'train_runtime': 914.0347, 'train_samples_per_second': 10.164, 'train_steps_per_second': 0.645, 'total_flos': 0.0, 'train_loss': 0.6522459773694055, 'epoch': 10.0})

In [ ]:
# Save the trained model
model.save_pretrained("./my_model")

# Push the trained model to the Huggingface Hub
model.push_to_hub("my-username/my-model", use_auth_token="my-token")

# Learn then test procedure & split conformal prediction

In this section we want to implement a conformal risk control method, that says that

**Conformal risk control theorem:**
Let $(x_{m+1}, y_{m+1}),\ldots, (x_{m+n}, y_{m+n})$ from the calibration set and $(x_{t}, y_{t})$ be i.i.d. samples from some distribution. Then, we can use the Learn the Test procedure to find a threshold $\alpha$ that ensures that
$$
    \mathbb{P}( \mathbb{E}(\mathcal{L}(C_\alpha(x_i),y_i)) \leq \lambda) \geq 1-\delta,
$$
for some desired values of $\lambda, \delta$.

In [ ]:
lambda_ = 2
delta = 0.05
epsilon = 0.05

In [ ]:
from transformers import pipeline
from tqdm import tqdm

classifier = pipeline("sentiment-analysis", model="summaries-comparisons-distilbert-TLDR", tokenizer=tokenizer, return_all_scores=True)

#todo: convert into a function that can be deployed easily for the test dataset.
def get_scores(X_cal):
  scores = dict(zip(['LABEL_'+str(i) for i in range(9)],[[] for _ in range(9)]))
  for e in tqdm(range(len(X_cal.index))):
    s = classifier(X_cal['text'][e])[0]
    dic = {}
    for d in s:
      dic[d['label']] = d['score']
    for k, v in dic.items():
      scores[k].append(v)
  return scores

scores = get_scores(X_cal)


In [ ]:
len(X_cal),len(X_cal.index), len(scores['LABEL_0'])
for i in range(9):
  X_cal['LABEL_'+str(i)] = scores['LABEL_' +str(i)]

In [ ]:
X_cal.head()

We can now compute the loss, at a given value of $\alpha$.

In [ ]:
# Define conformal_loss
def compute_loss(X_cal, alpha):
  numerator = 0
  denominator = 0
  for i in range(9):
    numerator += (i - X_cal['label'])**2 * X_cal['LABEL_'+str(i)] * (X_cal['LABEL_'+str(i)] > alpha)
    denominator += X_cal['LABEL_'+str(i)] * (X_cal['LABEL_'+str(i)] > alpha)
  X_cal['loss_alpha_'+str(alpha)] = numerator / denominator
  return X_cal

#X_cal = compute_loss(X_cal, alpha = 0.1)

In [ ]:
X_cal.head()

Finally we train the conformal model that fixes the value of 
$\alpha$ such that the prediction achieves error smaller than $\lambda$. To do so it implements the `Learn the Test' procedure which 
1. Generates a set of hypothesis.
2. Compute p-values according to $e^{-2n(\lambda- l)^2}$.
3. Correct the p-values using a family wise error correction.

For demonstration purpuses we will use $\lambda = 2$, with $\delta = 0.05$ too.

In [ ]:
def learn_the_test(compute_loss, X_cal, lambda_, delta):

    """ Computes the p-value of the hypothesis that the risk is lower than some \lambda."""

    pvalues = []

    # For each y
    alphas = np.linspace(0, 0.5, 51, endpoint = True)

    for alpha in alphas:

        # Compute the mean loss
        X_cal = compute_loss(X_cal, alpha = alpha)
        losses = [x for x in X_cal['loss_alpha_'+str(alpha)] if not np.isnan(x)
        loss = np.mean(losses)


        # Step 1: Compute p-values associated to such loss
        p_value = np.exp(-2*len(X_cal)*(max(lambda_ - loss, 0))**2)

        pvalues.append(p_value)

    # Step 2: Family-wise error correction
    reject, pvals_corrected, _, bonferroni_delta = multipletests(pvalues, delta, method = 'bonferroni')

    return alphas, pvals_corrected, reject

alphas, pvals_corrected, reject = learn_the_test(compute_loss, X_cal, lambda_ = lambda_, delta = delta)
for a, p, r in zip(alphas, pvals_corrected, reject):
  print("{:.2f}".format(a), "{:.4e}".format(p), r)

In [ ]:
alphas_ =  [alphas[i] for i in range(len(alphas)) if reject[i] == True and reject[i-1] == False] 
alpha = alphas_[0]
print(alpha)

Thus, we can use any of the values of $\alpha$ where the null hypothesis is rejected to satisfy the guarantees of the conformal risk control theorem.

In [ ]:
X_cal.head()

Once we have trained the language model, we compute the losses
$$\lambda_i = \mathcal{A}(z_1,\ldots,z_{train})(x_i,y_i),$$
where $x_i,y_i$ are elements of the calibration set. Similarly, we compute 
$$\lambda_t = \mathcal{A}(z_1,\ldots,z_{train})(x_t,y_t),$$
Here, we use the notation
$$ \mathcal{A}(z_1,\ldots,z_{train})(x_i,y_i) = \mathcal{L}(f(x_i),y_i).$$
We will use the conformal loss for some value of $\alpha$ where the risk is controlled such as $\alpha=0.6$.

The first way is to detect distribution shfit is compare the loss $\lambda_i$ in between the calibration and test sets, check whether 
$$
\mathbb{P}(\mathcal{L}(C_\alpha(x_i),y_i) \leq \lambda) \geq 1-\delta
$$
holds.

In [ ]:
# We first compute the loss in the shifted set at an alpha value of our choosing
scores = get_scores(X_shift)

len(X_shift),len(X_shift.index), len(scores['LABEL_0'])
for i in range(9):
  X_shift['LABEL_'+str(i)] = scores['LABEL_' +str(i)]

X_shift = compute_loss(X_shift, alpha = alpha)

cal_losses = list(X_cal['loss_alpha_'+str(alpha)])
shift_losses = list(X_shift['loss_alpha_'+str(alpha)])

In [ ]:
# Implement t-test
result = ttest_1samp(shift_losses, lambda_, alternative = 'greater')
if result.pvalue < epsilon:
  print('The losses in the test set violate the conformal risk control theorem, thus indicating a distribution shift with pvalue {}'.format(result.pvalue))
if result.pvalue < epsilon:
  print('The losses in the test set violate the conformal risk control theorem, thus indicating a distribution shift with pvalue {}'.format(result.pvalue))

Alternatively, we can check for statistically meaningful differences in the loss mean between the calibration and test sets

In [ ]:
shift_losses

In [ ]:
result = ttest_ind(cal_losses, shift_losses, equal_var = False, alternative = 'less')

pvalue = result.pvalue

if pvalue < epsilon:
  print('There is a difference in the mean loss between the calibration and test distributions, with pvalue {}'.format(pvalue))
else:
  print('pvalue is {}'.format(pvalue))

# Inductive conformal prediction

The second way is to use the properties of inductive conformal predictors. 

**Inductive conformal predictor**: An inductive conformal predictor $\Gamma^{\epsilon}(z_1,\ldots,z_m;z_{m+1},\ldots,z_{m+n})$ is defined as the set of possible values of $y$ for the test data point $x_t$ such that the corresponding $\lambda_t$ conforms to the rest of $\lambda_j$ of the calibration set:
$$
    \Gamma^{\epsilon}(z_1,\ldots,z_m;z_{m+1},\ldots,z_{m+n})(x_t) = \{y| p_t\geq \epsilon\},
$$
with the p-value
$$
    p_t = \frac{|\{j = m+1,\ldots,m+n|  \lambda_j \geq \lambda_t\}|+1}{l+1}.
$$

If we want $p_t\geq\epsilon$, this is approximately equivalent to finding an upper bound on $\lambda_t$ of the kind $\lambda_t = q_{1-\epsilon}(\{\lambda_j\})$, eg, the $1-\epsilon$ quantile of the $\{\lambda_j\}$ set. 
Inductive conformal predictors have the property that, if the calibration and test data are sampled i.i.d., then~\cite[Proposition 4.1]{vovk2005algorithmic}
$$
    \mathbb{P}(y_t\notin \Gamma^{\epsilon}(z_1,\ldots,z_m;z_{m+1},\ldots,z_{m+n})(x_t)\leq \epsilon).
$$

In [ ]:
# We first compute the loss in the shifted set at an alpha value of our choosing
scores = get_scores(X_shift)

len(X_shift),len(X_shift.index), len(scores['LABEL_0'])
for i in range(9):
  X_shift['LABEL_'+str(i)] = scores['LABEL_' +str(i)]

X_shift = compute_loss(X_shift, alpha = alpha)

cal_losses = list(X_cal['loss_alpha_'+str(alpha)])
shift_losses = list(X_shift['loss_alpha_'+str(alpha)])



In [ ]:
sorted_cal_losses = np.sort(cal_losses)

# Using again the same pvalue as above

lambda_upper_bound = sorted_cal_losses[int((1-epsilon)*(len(cal_losses)+1)-1)]
lambda_upper_bound = np.quantile(cal_losses, ((1-epsilon)*(len(cal_losses)+1)-1)/len(cal_losses))

lambda_upper_bound_simplified = np.quantile(cal_losses, 1-epsilon)

print(lambda_upper_bound, lambda_upper_bound_simplified, lambda_upper_bound_simplified)

Now we want to check whether
$$
    \mathbb{P}(y_t\notin \Gamma^{\epsilon}(z_1,\ldots,z_m;z_{m+1},\ldots,z_{m+n})(x_t)\leq \epsilon).
$$
holds according to a statistical test.

In [ ]:
# Implement binomial test
shift_losses_compare = np.array(shift_losses) < lambda_upper_bound
result = binomtest(sum(shift_losses_compare), len(shift_losses), 1-delta, alternative = 'less')
if result.pvalue < epsilon:
  print('The losses in the test set violate the inductive theorem, thus indicating a distribution shift with pvalue {}'.format(result.pvalue))
else:
  print('The pvalue is {}'.format(result.pvalue))

In [ ]:
print(np.mean(cal_losses), np.mean(shift_losses))

# Full conformal prediction (not necessary)


To perform full conformal prediction we divide the data in three sets: training (used to train the language model), calibration (used to train the conformal model outputing $\alpha$) and finally the test set where shifts may have happened.

Once we have trained the language model, we compute the losses
$$\lambda_i = \mathcal{A}(z_1,\ldots,z_{test},\ldots,z_{n_{cal}})(x_i,y_i),$$
where $x_i,y_i$ are elements of the calibration set, and 
$$ \mathcal{A}(z_1,\ldots,z_{test},\ldots,z_{n_{cal}})(x_i,y_i) = L(f(x_i),y_i),$$ 
where the calibration model $f$ was trained over $z_1,\ldots,z_{test},\ldots,z_{n_{cal}}$. Similarly is done to compute $\lambda_y$, which uses $L(f(x_{test}), y_{test})$, with the calibration model trained over the calibration set.

Once we have this, we compute the values of $y$ for with the p-value
$$p^y:= \frac{|\{i= m+1,\ldots,l|\lambda_i\leq \lambda_y\}|+1}{l-m+1}$$
is greater than $1-\epsilon$.

Finally, we would like to perform a series of binomial tests, with different calibration tests, to try to reject the null hypothesis that
$$\mathbb{P}_{X,Y}^l\left(\mathbb{P}_{X,Y} (y\in \Gamma(z_1,\ldots,z_l)(x))\geq 1-\epsilon\right)\geq 1-\delta$$
where $z_i = (a_i, b_i), y_i \in X_{train+calib}, Y_{train+calib}$